<h1>RAG System for CSV</h1>

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd


<h2>Import Data</h2>

The data was acqired from https://demo.oshinit.com/pages/mortgage-calculator.html. 300000 for amount, 360 months and Interest rate of 5%.

In [2]:
file_path = './data/mortage-300h-360-5.csv'

data = pd.read_csv(file_path, sep=';', encoding='utf-8')

data.head()

,Period,Monthly Payment,Computed Interest Due,Principal Due,Principal Balance
0,0,0.000000,0.000000,0.000000,300000.000000
1,1,1610.464869,1250.000000,360.464869,299639.535131
2,2,1610.464869,1248.498063,361.966806,299277.568325
3,3,1610.464869,1246.989868,363.475001,298914.093324
4,4,1610.464869,1245.475389,364.989480,298549.103844


load and process data

In [3]:
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ";",
    },
    encoding="windows-1252",
)

try:
    docs = loader.load()
except Exception as e:
    print("Error during data loading:", e)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(docs)
print(chunks[0])

# for record in docs[:2]:
#     print(record)
# documents =  [Document(page_content=text) for text in docs]
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
# chunks = text_splitter.split_documents(documents)



page_content='Period: 0
Monthly Payment: 0
Computed Interest Due: 0
Principal Due: 0
Principal Balance: 300000' metadata={'source': './data/mortage-300h-360-5.csv', 'row': 0}


: 

In [ ]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="mistral:latest"),
    collection_name="local-rag"
)

In [ ]:
# LLM from Ollama
local_model = "llama3.1:latest"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five question-answering variations of the given user question to retrieve relevant documents from a vector database. By framing the query as potential answers to a question, your goal is to identify documents that directly address the user's information need. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What is the total monthly payment?")